In [56]:
# gonna start off with a whole bunch of imports; we can always get rid of ones 
# we don't need
import numpy as np
import matplotlib.pyplot as plt
import math
import numpy.testing as npt
from time import time
import scipy.linalg as LA
from numba import cuda
#from simple_plot import *

First, lets start with attempting to get a parallel function to do value iteration for a small MDP and grid

In [57]:
# the ratio of a circle's circumference to its diameter, aka, three and a bit
PI = np.pi
# threads per block
TPB = 8

@cuda.jit(device = True)
def d_isActionAllowed(x_curr, y_curr, action, nx, ny, wall_coords, term_coords):
    """
    wall_coords is an n by 2 array of wall coordinate values
    term_coords is an m by 2 array of terminal state coordinate values
    """
    # initialize the isAllowed boolean to true: 
    isAllowed = True
    x_new = -1
    y_new = -1

    if action == STAY:
        x_new = x_curr
        y_new = y_curr
    elif action == UP:
        x_new = x_curr
        y_new = y_curr + 1
    elif action == RIGHT:
        x_new = x_curr + 1
        y_new = y_curr
    elif action == DOWN:
        x_new = x_curr
        y_new = y_curr -1
    elif action == LEFT:
        x_new = x_curr -1
        y_new = y_curr

    # first check to make sure the new move is in-bounds for the grid: 
    if x_new >= nx or x_new < 0 or y_new >= ny or y_new < 0:
        isAllowed = False
    
    # then check to see if you're in a terminal state ...
    isTerminal = False
    for i in range(term_coords.shape[0]):
        if term_coords[i, 0] == x_curr and term_coords[i, 1] == y_curr:
            isTerminal = True
    # the agent *must* stay if they are in a terminal state:
    if action != STAY and isTerminal == True: 
        isAllowed = False
    # the agent is not allowed to *choose* to stay unless they are in a terminal state: 
    elif action == STAY and isTerminal == False: 
        isAllowed = False

    # then check to make sure your new move isn't into a wall: 
    for i in range(wall_coords.shape[0]):
        if wall_coords[i, 0] == x_curr and wall_coords[i, 1] == y_curr:
            isAllowed = False

    return isAllowed

#kernel accessing global memory array
@cuda.jit
def valueIteration_par_kernel(d_u, d_p, d_r, discount, nx, ny, d_wc, d_tc, parity):
    """
    perform value iteration, i hope
    
    Arguments:
        
    returns:
        None
    """
    i, j = cuda.grid(2)
    dims = d_u.shape

    # return if we're out of bounds
    if i >= (dims[0]) or j >= (dims[1]):
        return

    # return if we're in a terminal state: 
    for k in range(d_tc.shape[0]):
        if d_tc[k, 0] == i and d_tc[k, 1] == j:
            return
    
    # return if we're inside a wall: 
    #for m in range(d_wc.shape[0]):
    #    if d_wc[m, 0] == i and d_wc[m, 1] == j:
    #        return

    # We define black squares as squares where i+j is even, and red squares 
    # as squares where i+j is odd. Furthermore, we define our parity such that black 
    # squares have parity == 0 while red squares have parity == 1.
    # 
    # If (i+j)%2 == 1, then (i+j) is odd, so we're at a red square, so we should only 
    # calculate d_u[i, j] if parity == 1. Conversely, if (i+j)%2 == 0, then (i+j) is 
    # even, and we're at a black square, so we should only calculate d_u[i, j] if 
    # parity == 0. This simplifies down to the following if-statement: 
    if (i+j)%2 == parity:
        maxQVal = -666420.0
        for a in range(NUM_ACTIONS): 
            # begin calculation for current qValue ===========================
            qValue = np.float32(0.0)
            for sP in range(NUM_ACTIONS): 
                if d_isActionAllowed(i, j, a, nx, ny, d_wc, d_tc): 
                    # calculate the probability and reward for each action-
                    # state_prime pair: 
                    uPrime = 0
                    prob = d_p[i, j, a, sP]
                    reward = d_r[i, j, a, sP]
                    if sP == STAY:
                        uPrime = d_u[i, j]
                    elif sP == UP and j+1 < (dims[1]): 
                        uPrime = d_u[i, j+1]
                    elif sP == DOWN and j-1 >= 0: 
                        uPrime = d_u[i, j-1]
                    elif sP == LEFT and i-1 >= 0: 
                        uPrime = d_u[i-1, j]
                    elif sP == RIGHT and i+1 < (dims[0]): 
                        uPrime = d_u[i+1, j]
                    
                    qValue += prob*(reward + discount*uPrime)
            # end calculation for current qValue ===========================
            curr_qVal = qValue
            # determine if current qValue is the new maximum qValue: 
            if curr_qVal >= maxQVal:
                maxQVal = curr_qVal
        d_u[i, j] = maxQVal


def valueIteration_par(u2, p2, r2, iters, discount, nx, ny, wall_coords, term_coords):
    """
    Wrapper function for computing q-values
    
    Arguments:
        u: numpy float 2D array of current values
        iters: int number of update iterations
    Returns:
        updated 2D array of values
    """
    start = cuda.event()
    end = cuda.event()
 
    wc_length = len(wall_coords)
    wc = np.zeros(shape=[wc_length, 2], dtype = int)
    i = 0
    for wc_tuples in wall_coords:
        wc[i, 0] = wc_tuples[0]
        wc[i, 1] = wc_tuples[1]
        i += 1
    tc_length = len(term_coords)
    tc = np.zeros(shape=[tc_length, 2], dtype = int)
    j = 0
    for tc_tuples in term_coords:
        tc[j, 0] = tc_tuples[0]
        tc[j, 1] = tc_tuples[1]
        j += 1

    d_u = cuda.to_device(u2)
    d_p = cuda.to_device(p2)
    d_r = cuda.to_device(r2)
    d_wc = cuda.to_device(wc)
    d_tc = cuda.to_device(tc)
    dims = d_u.shape
    gridDims = [(dims[0]+TPB-1)//TPB, (dims[1]+TPB-1)//TPB]
    blockDims = [TPB, TPB]

    start.record()
    for k in range(iters):
        valueIteration_par_kernel[gridDims, blockDims](d_u, d_p, d_r, discount, nx, ny, d_wc, d_tc, 0)
        valueIteration_par_kernel[gridDims, blockDims](d_u, d_p, d_r, discount, nx, ny, d_wc, d_tc, 1)

    end.record()
    end.synchronize()
    elapsed = cuda.event_elapsed_time(start, end)

    return elapsed, d_u.copy_to_host()

In [58]:
def valueIteration_ser(u, p, r, iterations, discount, nx, ny, wall_coords, term_coords):
    dims = u.shape
    nx = dims[0]
    ny = dims[1]

    u_new = np.copy(u)
    
    for n in range(iterations):
        u_old = np.copy(u_new)
        for i in range(nx):
            for j in range(ny):
                # we don't iterate for terminal states or inside of walls: 
                if ((i, j) in term_coords) == True or ((i, j) in wall_coords):
                    continue
                maxQVal = -666420.0 
                for a in range(NUM_ACTIONS): 
                    # begin calculation for current qValue ===========================
                    qValue = np.float32(0.0)
                    for sP in range(NUM_ACTIONS): 
                        if isActionAllowed((i, j), a, nx, ny, wall_coords, term_coords): 
                            # calculate the probability and reward for each action-
                            # state_prime pair: 
                            uPrime = np.float32(0.0)
                            prob = p[i, j, a, sP]
                            reward = r[i, j, a, sP]
                            if sP == STAY:
                                uPrime = u_old[i, j]
                            elif sP == UP and j+1 < (dims[1]): 
                                uPrime = u_old[i, j+1]
                            elif sP == DOWN and j-1 >= 0: 
                                uPrime = u_old[i, j-1]
                            elif sP == LEFT and i-1 >= 0: 
                                uPrime = u_old[i-1, j]
                            elif sP == RIGHT and i+1 < (dims[0]): 
                                uPrime = u_old[i+1, j]
                            
                            qValue += prob*(reward + discount*uPrime)
                    # end calculation for current qValue ===========================
                    curr_qVal = qValue
                    # determine if current qValue is the new maximum qValue: 
                    if curr_qVal >= maxQVal:
                        maxQVal = curr_qVal
                u_new[i, j] = maxQVal
    
    return u_new


In [59]:
def isActionAllowed(curr_coords, action, nx, ny, wall_coords, term_coords):
    """
    curr_coords is a tuple of ints (x_coord, y_coord)
    wall_coords is a list of tuples of ints
    """
    # initialize the isAllowed boolean to true: 
    isAllowed = True
    x_curr = curr_coords[0]
    y_curr = curr_coords[1]
    x_new = -1
    y_new = -1

    if action == STAY:
        x_new = x_curr
        y_new = y_curr
    elif action == UP:
        x_new = x_curr
        y_new = y_curr + 1
    elif action == RIGHT:
        x_new = x_curr + 1
        y_new = y_curr
    elif action == DOWN:
        x_new = x_curr
        y_new = y_curr -1
    elif action == LEFT:
        x_new = x_curr -1
        y_new = y_curr

    # first check to make sure the new move is in-bounds for the grid: 
    if x_new >= nx or x_new < 0 or y_new >= ny or y_new < 0:
        isAllowed = False
    
    # then check to make sure you're not at a terminal state: 
    if (x_curr, y_curr) in term_coords and action != STAY: 
        isAllowed = False
    
    # the agent is not allowed to *choose* to stay unless they are in a terminal state: 
    if action == STAY and ((x_curr, y_curr) in term_coords) == False: 
        isAllowed = False

    # then check to make sure your new move isn't into a wall: 
    if (x_new, y_new) in wall_coords: 
        isAllowed = False

    return isAllowed

# initialize the probability matrix (or is it a tensor?)
def probSetup(nx, ny, term_coords, wall_coords, noise):
    """
    takes nx, ny, coords for terminal states, the coordinates for interior walls, 
    noise, anything else? and returns p
    """
    # the signal is 1 minus the noise: 
    signal = 1.0 - noise
    # initialize the probability matrix to a bunch of zeros: 
    p_init = np.zeros(shape=[nx, ny, NUM_ACTIONS, sP_size], dtype = np.float32)

    # loop through all of the x and y values: 
    for i in range(nx):
        for j in range(ny):
            # loop through all the action values: 
            for a in range(NUM_ACTIONS): 
                # These statements are important for edge cases, e.g.: if the agent tries 
                # to move up, but both up and, say, left are blocked, then there should 
                # be a 10% chance to move right, and a (80% plus 10% =) 90% chance to 
                # stay. The edge cases are dealt with in the mess of if-else statements 
                # in the rest of this loop.
                upAllowed = isActionAllowed((i, j), UP, nx, ny, wall_coords, term_coords)
                rightAllowed = isActionAllowed((i, j), RIGHT, nx, ny, wall_coords, term_coords)
                downAllowed = isActionAllowed((i, j), DOWN, nx, ny, wall_coords, term_coords)
                leftAllowed = isActionAllowed((i, j), LEFT, nx, ny, wall_coords, term_coords)

                if isActionAllowed((i, j), a, nx, ny, wall_coords, term_coords): 
                    # this if-statement is only triggered for terminal states (the 'stay' 
                    # action is not allowed for non-terminal states): 
                    if a == 0: 
                        # the probability of staying/exiting when the action 'stay/exit' 
                        # is selected is always 1 for terminal states
                        p_init[i, j, STAY, STAY] = 1.0

                    # if you don't stay, then signal% chance of performing the action you take: 
                    p_init[i, j, a, a] = signal

                    # if the action is left or right, then you have a [noise/2]% chance 
                    # to go up, and the same to go down: 
                    if a == LEFT or a == RIGHT:
                        if upAllowed: 
                            p_init[i, j, a, UP] = noise/2.0
                        else: 
                            # the '+=' here is b/c if the agent is in a corridor, then 
                            # the prob of STAY ends up being (noise/2) + (noise/2)
                            p_init[i, j, a, STAY] += noise/2.0
                        if downAllowed: 
                            p_init[i, j, a, DOWN] = noise/2.0
                        else: 
                            p_init[i, j, a, STAY] += noise/2.0
                    # if the action is up or down, then you have a [noise/2]% chance to 
                    # go left, and the same to go right: 
                    else: # the action
                        if rightAllowed: 
                            p_init[i, j, a, RIGHT] = noise/2.0
                        else: 
                            p_init[i, j, a, STAY] += noise/2.0
                        if leftAllowed: 
                            p_init[i, j, a, LEFT] = noise/2.0
                        else: 
                            p_init[i, j, a, STAY] += noise/2.0
                # if !isActionAllowed, then we gotta update the STAY probabilities: 
                else: 
                    if a > 0: 
                        if a == LEFT or a == RIGHT:
                            if upAllowed: 
                                p_init[i, j, a, UP] = noise/2.0
                            else: 
                                p_init[i, j, a, STAY] += noise/2.0
                            if downAllowed: 
                                p_init[i, j, a, DOWN] = noise/2.0
                            else: 
                                p_init[i, j, a, STAY] += noise/2.0
                            p_init[i, j, a, STAY] += signal
                        # a == UP or a == DOWN: 
                        else: 
                            if rightAllowed: 
                                p_init[i, j, a, RIGHT] = noise/2.0
                            else: 
                                p_init[i, j, a, STAY] += noise/2.0
                            if leftAllowed: 
                                p_init[i, j, a, LEFT] = noise/2.0
                            else: 
                                p_init[i, j, a, STAY] += noise/2.0
                            p_init[i, j, a, STAY] += signal
    
    return p_init
                
    
    

In [60]:
# here we're gonna set up the MDP and grid:
nx, ny = 100, 100
# the actions are: 0 = stay/exit, 1 = up, 2 = right, 3 = down, 4 = left. Note that up and 
# down are odd, while left and right are even; this is convenient because an odd action 
# has a [noise]% chance of taking an even action, and vice versa
STAY = 0
UP = 1
RIGHT = 2
DOWN = 3
LEFT = 4
NUM_ACTIONS = 5 
sP_size = 5 
iters = 500
discount = np.float32(0.9)
# probability of performing the action you're attempting to perform: 
signal = 0.8
# probability of doing not that: 
noise = 1.0 - signal

# initialize 
term_coords = [(nx-1, ny-1), (0, ny-1)]
wall_coords = [(-1, -1)]
# lets have a bunch of equally spaced walls, just to make things spicy
for i in range(nx):
    for j in range(ny):
        if i%3 == 0 and j %3 == 0 and ((i, j) in term_coords) == False: 
            wall_coords.append((i, j))

p_init = probSetup(nx, ny, term_coords, wall_coords, noise)
u_init = np.zeros(shape=[nx, ny], dtype = np.float32)
r_init = np.zeros(shape=[nx, ny, NUM_ACTIONS, sP_size], dtype = np.float32)

for i in range(nx):
    for j in range(ny):
        for a in range(NUM_ACTIONS): 
            for b in range(NUM_ACTIONS):
                # for basic bitch grid, all actions (except STAY in a terminal state) 
                # have a reward of WHATEVER I WANT (they have a reward of zero and a 
                # discount of 0.9 in order to be consistent with the Berkeley pacman 
                # AI projects): 
                r_init[i, j, a, b] = 0.0

# now we take care of the terminal states; first, we give the reward for staying in a 
# terminal state, as well as the u value for the terminal states: 
u_init[nx-1, ny-1] = -20.0
u_init[0, ny-1] = 20.0
r_init[nx-1, ny-1, STAY, STAY] = -20.0
r_init[0, ny-1, STAY, STAY] = 20.0

# we gotta copy the initial matrices and send the copies to the valueIteration functions 
# so that the initial matrices don't get altered 
u1 = np.copy(u_init)
p1 = np.copy(p_init)
r1 = np.copy(r_init)
start = time()
u_ser = valueIteration_ser(u1, p1, r1, iters, discount, nx, ny, wall_coords, term_coords)
stop = time()
serial_elapsed = stop - start

u2 = np.copy(u_init)
p2 = np.copy(p_init)
r2 = np.copy(r_init)
parallel_elapsed, u_par = valueIteration_par(u2, p2, r2, iters, discount, nx, ny, wall_coords, term_coords)

print("serial_elapsed =", serial_elapsed, "seconds")
print("parallel_elapsed =", parallel_elapsed/1000, "seconds")

npt.assert_allclose(u_ser, u_par, atol = 1e-2)


serial_elapsed = 1625.3669016361237 seconds
parallel_elapsed = 4.88345703125 seconds
